# Projeto 2 - Ciência dos Dados - Estudo sobre Ifood

Nome: Lucas Hix

Nome: Andrea Mindlin Tessler

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
# Importações

import datetime
import tweepy
import math
import os.path
import pandas as pd
import json
import re
import string
from random import shuffle

___
## Autenticando no  Twitter

* Conta - Lucas Hix: ***@Peng1104Oficial***
* Conta - Andrea Tessler: ***@Andrea25519779***

In [2]:
# Dados de autenticação do twitter:

def mkAPI(filename):
    # leitura do arquivo no formato JSON
    
    with open(filename) as file:
        data = json.load(file)
    
    # Configurando a biblioteca. Não modificar
    
    auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
    auth.set_access_token(data['access_token'], data['access_token_secret'])
    
    # Cria um objeto de API
    return tweepy.API(auth)

# Conta @Peng1104Oficial
api_lucas = mkAPI("lucas-auth.pass")

# Conta @Andrea25519779
api_andrea = mkAPI("andrea-auth.pass")

#Lista contendo as APIs
apis = [api_lucas, api_andrea]

___
## Etapas do projeto:

### Funções essenciais para o projeto

In [13]:
# Variaveis de suporte para a captura de Tweets
ultimo_id =-1
ultima_data = datetime.datetime.now()

# Tempo para atualização do ultimo_id (em minutos)
tempo_de_reset_do_ultimo_id = 60

# Função para a captura de Tweets
def getTweets(palavra_de_procura, quantidade_de_procuras_por_api, ultimo_id, ultima_data):
    # Lista do resultado do final da captura (os tweets selecionados)
    resultado = []
    
    # Numero da API que ta sendo usada (uso para debug)
    numero_api = 0
    
    # Captura o numero de tweets adicionados por API
    resultado_api = {}
    
    # Renicia o ultimo_id para voltar a pegar tweets recentes
    
    if ultima_data <= datetime.datetime.now():
        ultima_data = datetime.datetime.now() + datetime.timedelta(minutes=tempo_de_reset_do_ultimo_id)
        ultimo_id = -1
        
        print("AVISO: Houve um reinicio do ultimo_id, data_atual =", ultima_data)
    
    # Para cada API gerada
    for api in apis:
        numero_api = numero_api + 1
        
        # Contagem de captura para a API
        contagem = 0
        
        # Inicia a contagem para a API
        resultado_api[numero_api] = 0
        
        if ultimo_id < 0:
            cursor = tweepy.Cursor(api.search, q=palavra_de_procura, lang="pt", tweet_mode="extended")
        else:
            cursor = tweepy.Cursor(api.search, q=palavra_de_procura, lang="pt", tweet_mode="extended", max_id=ultimo_id)
        
        # Captura de um novo tweet
        for tweet in cursor.items(quantidade_de_procuras_por_api):
            
            # Texto do tweet capturado
            texto = tweet.full_text.lower()
                
            # Verfica se a palavra de procura se encontra no texto:
            if palavra_de_procura.lower() in texto:
                    
                # Não adiciona tweet duplicado
                if texto not in resultado:
                        
                    # Adiciona o tweet ao resultado
                    resultado.append(texto)
                        
                    # Adiciona as capturas da api + 1
                    resultado_api[numero_api] = resultado_api[numero_api] + 1
                
            # Adiciona 1 captura a contagem de capturas para esta API
            contagem = contagem + 1
            
            # Debug para saber como esta indo o código
            print("API {0} de {1} - {2:.0f}% das capturas realizadas, {3} tweets validos".format(
                  numero_api, len(apis), (contagem/quantidade_de_procuras_por_api)*100, len(resultado)), end="\r")
            
            # Captura o ultimo id da captura para não duplicar pedido
            if contagem == quantidade_de_procuras_por_api:
                ultimo_id = tweet.id
            
    # Embaralhando as mensagens para reduzir um possível viés
    shuffle(resultado)
    
    # Mensagem debug
    for api, quantidade in resultado_api.items():
        print("A API {0} capturou {1} tweets de {2}, equivalente a {3:.2f}% do total".format(api, quantidade, len(resultado), quantidade/len(resultado)*100))
    
    return resultado, ultimo_id, ultima_data

# Função para limpar um texto de um Tweet, poder avaliar o menos sem pontuações links e menções de nomes assim como a palavra ultilizada na captura
def limpar_texto(texto, marca):
    # Remove Menções
    texto = re.sub("@[^ ]*", "", texto)
    
    # Remove links
    texto = re.sub("http(|s):\/\/[^ \n]*", "", texto)
    
    # Remove Pontuações
    texto = re.sub("[" + string.punctuation + "]+", "", texto)
    
    # Remove a marca analisada
    return re.sub(marca.lower(), "", texto)

### Escolha de uma marca e coleta das mensagens pela primeira vez

Separação da marca a ser analisada e a quantiade de capturas a ser feitas para cada api

In [4]:
# Escolha da marca a ser analisada
marca = "Ifood"

# Esolha de quantidade por api (no caso quantidade*2)
quantidade_por_api = 500

Preparação para a nomeação dos Excels

In [5]:
# Numero de capturas ja realizadas
numero_de_capturas_realizadas = 1

# Verifica dentro do diretorio o numero de capturas realizadas
for file in os.listdir("."):
    if marca in file:
        numero_de_capturas_realizadas = numero_de_capturas_realizadas + 1

# Cria o nome do excel a ser criado
nome_base_do_arquivo = marca + "-" + str(numero_de_capturas_realizadas) 

nome_do_arquivo = nome_base_do_arquivo + ".xlsx"

Captura dos dados iniciais

In [6]:
#dados_iniciais, ultimo_id, ultima_data = getTweets(marca, quantidade_por_api, ultimo_id, ultima_data)

Salva os dados iniciais capturados em um Execel

In [7]:
#tabela = pd.DataFrame({"Tweets": pd.Series(dados_iniciais), "Categoria": [""]*len(dados_iniciais)})
#tabela.to_excel(nome_do_arquivo, index = False)

#print("O Excel", nome_do_arquivo, "foi gerado!")

___
### Classificando as mensagens

Esta etapa é manual, feita no Excel.

___
### Montando o Classificador Naive-Bayes

Ensinando o classificador.

In [8]:
# Transforma os excels avaliados em um Classificador

def criar_naive_bayes(excels, marca):
    # Cria o DataFrame da tabela de treinamento
    
    # Coluna Tweets -> Mensagens Capturadas do Twitter
    # Coluna Categoria -> Coluna que foi avaliada manualmente, definindo a categoria do tweet
    # Coluna Categoria atribuida automaticamente -> Pode ou não existir irelevante para o Naiv Bayes -> Ver: @1
    # 
    # @1 -> Caso fosse ultilizado irira criar uma tendencia de "vicio", criando um loop de incertezas no classicador
    
    # DataFrame Inicial para o uso do pd.contact
    tabela_treinamento = pd.DataFrame({"Tweets": [""], "Categoria": [""],
                                      "Categoria atribuida automaticamente": [""]})
    
    # Junta todas as tabelas de treinamento em uma
    for tabela in excels:
        tabela_treinamento = pd.concat([tabela_processada, pd.read_excel(tabela)], sort=False)
    
    # Remove o DataFrame incial (Agora so tem os dados avaliados)
    tabela_treinamento = tabela_treinamento.iloc[1:]
    
    tamanho_original = len(tabela_treinamento)
    
    # Remove todas os Tweets duplicados da tabela conjuta de treinamento
    tabela_treinamento = tabela_treinamento.drop_duplicates(subset="Tweets")
    
    print("Foram removidas {0} linhas duplicadas para a formação do Naive Bayes".format(tamanho_original-len(tabela_treinamento)))
    
    # Limpa todos os teewts e cria uma lista contendo todas as palavras da tabela de treinamento
    palavras = limpar_texto(tabela_treinamento["Tweets"].str.cat(), marca).split()
    
    # Cria a variavel de apoio do Laplace n2
    todas_as_palavras = set(palavras)
    
    # Variavel contendo todas as categorias definidas
    categorias = set(tabela_treinamento["Categoria"])
    
    # Dicionario resultado contendo a categoria como chave e uma lista contendo os elementos para o lapas
    dict_resultado = {}
    
    for categoria in categorias:
        # Cria uma sub tabela contendo apenas os dados de uma categoria
        tabela_categoria = tabela_treinamento[tabela_treinamento["Categoria"] == categoria]
        
        # Juta todas os Tweets da categoria em uma string ultilizando a função limpar_texto
        mensagens = limpar_texto(tabela_categoria["Tweets"].str.cat(), marca)
        
        # Cria uma series contendo todas as palavras da categoria
        series = pd.Series(mensagens.split())
        
        # Cria a probabilidade da palavra pertencer a categoria
        prior = len(tabela_categoria.index)/len(tabela_treinamento.index)
        
        print("O prior da categoria {0} é de {1:.2f}%".format(categoria, prior*100))
        
        dict_resultado[categoria] = [series.value_counts(), prior]
    
    return dict_resultado, todas_as_palavras

___
### Verificando a performance

Função para teste do naive_bayes

In [9]:
# Função para executar um teste dando novos tweets ultilizado um dicionanrio da naive_bayes

def executar_teste(novos_tweets, dict_de_atribuicoes, todas_as_palavras, marca):
    # Lista contendo os resultados obtidos automaticamente para cada tweet novo na sequencia dos mesmos
    resultado = []
    
    #Separa cada tweet nos novos tweets
    for tweet in novos_tweets:
        
        # Aramazena as probabilidades de cada categoria no dicionario para o tweet que esta sendo analisado
        probabilidades = {}
        
        # Separa cada palavra no tweet a ser analisado
        for palavra in limpar_texto(tweet, marca).split():
            
            # Para cada tipo de categoria
            for categoria, dados_categoria in dict_de_atribuicoes.items():
                
                # A series desta categoria contendo a contagem de todas as palavras pertencendo a esta categoria
                series_da_categoria = dados_categoria[0]
                
                # Ultiliza Laplace para calcular a probabilidade da palavra pertencer a essa categoria
                if palavra in series_da_categoria:
                    # Probabilidade = P(palavra|categoria) + 1 / quantiade de palavras na categoria + a quantidade de todas as palavras na tabela de treinamento
                    probabilidade = (series_da_categoria[palavra] + 1) / (len(series_da_categoria) +  len(todas_as_palavras))
                else:
                    # Caso a palavra não pertence a categoria calcula a probabilidade da mesma pertencer
                    # Probabilidade = 0 + 1 / quantiade de palavras na categoria + a quantidade de todas as palavras na tabela de treinamento
                    probabilidade = 1 / (len(series_da_categoria) + len(todas_as_palavras))
                
                # Adiciona a probabilidade da palavra ao conjunto das probabilidades
                if categoria in probabilidades:
                    # Adiciona a probabiliade da palavra a probabilidade da categoria ja existente 
                    probabilidades[categoria] = probabilidades[categoria] * probabilidade
                else:
                    # Cria a primeira probabiliade da categoria que é igual a P(palavra) + P(categoria)
                    probabilidades[categoria] = probabilidade * dados_categoria[1]
        
        # Adiciona ao resultado o valor atribuido a esta mensagem
        
        # Deu algum erro
        if len(probabilidades) == 0:
            resultado.append(-1)
        else:
            # Ultilizando o dicionario das probabilidades, procura a maior probabilidade e adicona a categoria que a represneta (o Naiv Bayes)
            resultado.append(max(probabilidades, key=probabilidades.get))
    
    return resultado

___
### Ultilizando as funções

Testando todas as funções criadas

In [12]:
# Numero de tentativas já feitas, 
Numero_de_tentativas = 5

In [13]:
def run_all(tabelas_avaliadas, ultimo_id, ultima_data, Numero_de_tentativas):
    # Adiciona esta tentativa
    Numero_de_tentativas = Numero_de_tentativas + 1
    
    # Captura novos tweets
    novos_tweets, ultimo_id, ultima_data = getTweets(marca, quantidade_por_api, ultimo_id, ultima_data)

    # Cria o classifcador ultizando todas as tabelas avalidas
    classificador, todas_as_palavras = criar_naive_bayes(tabelas_avaliadas, marca)

    # Cria a auto_avaliação feita pelo classficador (Naive Bayes)
    auto_avaliação_dos_novos_tweets = executar_teste(novos_tweets, classificador, todas_as_palavras, marca)

    # Cria o nome do excel a ser criado
    nome_do_arquivo_de_teste = marca + "-Teste-" + str(Numero_de_tentativas) + ".xlsx"

    # Cria a tabela que vai ser salva no excel contendo os novo tweets um espaço para a avaliação manual (Categoria)
    # e os valores atribuidos automaticamente pelo classifador em (Categoria atribuida automaticamente)
    tabela = pd.DataFrame({"Tweets": pd.Series(novos_tweets), "Categoria": [""]*len(novos_tweets),
                           "Categoria atribuida automaticamente": auto_avaliação_dos_novos_tweets})

    # Savalva a tabela no excel
    tabela.to_excel(nome_do_arquivo_de_teste, index=False)

    # Finaliza o teste
    print("O Excel", nome_do_arquivo_de_teste, "foi gerado!")
    
    return ultimo_id, ultima_data, Numero_de_tentativas

In [14]:
# Escolha da marca a ser analisada
marca = "Ifood"

# Esolha de quantidade por api (no caso quantidade*2)
quantidade_por_api = 250

# Lista das tabelas ja avaliadas
tabelas_avaliadas = []

tabelas_avaliadas.append("Ifood-Original.xlsx")
tabelas_avaliadas.append("Ifood-Teste-2.xlsx")
tabelas_avaliadas.append("Ifood-Teste-3.xlsx")
tabelas_avaliadas.append("Ifood-Teste-4.xlsx")


# Executa tudo e gera um novo execel de teste
ultimo_id, ultima_data, Numero_de_tentativas = run_all(tabelas_avaliadas, ultimo_id, ultima_data, Numero_de_tentativas)

AVISO: Houve um reinicio do ultimo_id, data_atual = 2019-09-17 17:09:51.917209
A API 1 capturou 227 tweets de 422, equivalente a 53.79% do total
A API 2 capturou 195 tweets de 422, equivalente a 46.21% do total
Foram removidas 3 linhas duplicadas para a formação do Naive Bayes
O prior da categoria CUPOM é de 9.64%
O prior da categoria RUIM é de 34.01%
O prior da categoria IMPOSSIVEL SABER é de 3.12%
O prior da categoria IRRELEVANTE é de 18.78%
O prior da categoria BOM é de 34.45%
O Excel Ifood-Teste-6.xlsx foi gerado!


___
### Conclusões


Após varias tabelas avaliadas acreditamos que poderia ser feito melhorias como atribuir um valor especial para palavras caracteristicas de uma categoria como a identificação de # nos twitters.


## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**